<a href="https://colab.research.google.com/github/AnnaZhuravleva/AOT/blob/master/HW3/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 3. Разрешение семантической многозначности.

При выполнении домашнего задания можно пользоваться тетрадками с семинаров.

### Описание задания:

Выберите одну из предложенных лексем: _свет_, _острый_, _лук_, _замок_.

1. Выберите 2-3 значения лексемы по словарю (рекомендую использовать [МАС](http://feb-web.ru/feb/mas/mas-abc/), обязательно напишите, каким словарем пользовались).
Для каждого значения придумайте несколько "диагностических" слов (которые сигнализируют о реализации одного из значений в контексте): 
например, _ключ_ — _вода_ / _дверь_. Подберите по НКРЯ контексты для выбранной лексемы, содержащие диагностические слова.
Таким образом, получается размеченный тестсет.

2. Для каждого контекста определите значение автоматически — с помощью AdaGram / ELMo на выбор.
В случае с AdaGram вы сразу получаете какой-то номер значения, с ELMo — нужно кластеризовать.

3. Возьмите словарные толкования для каждого значения и припишите ему значение автоматически
тем же способом, что и в п.2.

4. Придумайте и реализуйте способ соотнести толкования с значениями с помощью тезауруса (Wiki_ru_wordnet / РуТез). 

5. Оцените accuracy — в какой доле контекстов автоматически определенно значение для контекста совпадает 
с автоматически определенным значением толкования.

### Критерии оценки:

по 2 балла на каждый пункт

### Формат сдачи задания:

Jupyter-notebook на гитхабе.

### Дедлайн: 

9 декабря 2019 10:00мск



In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
elmo_path = '/content/drive/My Drive/Colab Notebooks/nlp'
project_path = '/content/drive/My Drive/Colab Notebooks/nlp'
sys.path.append(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import urllib
# urllib.request.urlretrieve("https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib", "/content/drive/My Drive/Colab Notebooks/nlp/all.a010.p10.d300.w5.m100.nonorm.slim.joblib")
!pip install git+https://github.com/lopuhin/python-adagram.git
!pip install pymorphy2
import adagram
# from allennlp.commands.elmo import ElmoEmbedder
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import tqdm
import nltk
nltk.download('stopwords')
from sklearn import metrics
from collections import Counter
import numpy as np

!pip install wiki-ru-wordnet
from wiki_ru_wordnet import WikiWordnet
wikiwordnet = WikiWordnet()

morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')

stops = set(stopwords.words('russian'))

def normalize_pm(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word not in stops]
    return words

def tokenize(text):
    return token.tokenize(text)

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-dulzwg2o
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-dulzwg2o
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=464615 sha256=68e4ce7dd6dd2e47b7acaadd0be729874a816e61135a7c61b2880c689fa58323
  Stored in directory: /tmp/pip-ephem-wheel-cache-m34uel07/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Выбранные значения:

1.1) Электромагнитное излучение, воспринимаемое глазом и делающее видимым окружающий мир; 

2) Земля со всем существующим на ней, мир, вселенная. 

3) Метония от значения (1) -> Источник освещения и приспособление для освещения в домах и на улицах

**Словарь - МАС**

#### Диагностические слова

1) *источник, солнце, луна, излучение, луч*

2) *сторона (света), белый (свет), часть (света), вокруг (света), край (света)*

3) *зажечь, погасить, включить, выключить*

In [0]:
with open('/content/drive/My Drive/Colab Notebooks/nlp/set.txt', 'r', encoding='utf-8-sig') as f:
    text = list(f.readlines())
    test_X = [t.split('\t')[0].strip() for t in text]
    test_y = [int(t.split('\t')[-1].strip()) for t in text]

In [0]:
vm = adagram.VectorModel.load('/content/drive/My Drive/Colab Notebooks/nlp/all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [5]:
for idx, sense in enumerate(vm.word_sense_probs('свет')):
    print(idx,[i[0] for i in vm.sense_neighbors('свет', sense[0])])

/usr/local/lib/python3.6/dist-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


0 ['луч', 'световой', 'рассеивать', 'излучение', 'спектр', 'излучать', 'монохроматический', 'интенсивность', 'радиоволна', 'фотон']
1 ['сияние', 'бледный', 'отблеск', 'неяркий', 'свечение', 'мерцание', 'отсвет', 'тусклый', 'блик', 'блик']
2 ['земля', 'похожий', 'походить', 'мир', 'планета', 'край', 'диво', 'сцена', 'способный', 'жениться']
3 ['появляться', 'переиздание', 'выходить', 'издание', 'книга', 'сборник', 'трехтомник', '1869', 'экран', 'журнал']
4 ['сияние', 'сиять', 'озарять', 'тьма', 'луч', 'истина', 'светоч', 'неизреченный', 'вечность', 'лучезарный']
5 ['лампа', 'ночник', 'погасить', 'зажигать', 'лампа', 'зажигаться', 'освещение', 'лампочка', 'дневной', 'фонарь']


Определяем значения для каждого контекста автоматически с помощью **adagram**

In [6]:
tested_y = [np.argmax(vm.disambiguate('свет', normalize_pm(item))) for item in test_X]

/usr/local/lib/python3.6/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [7]:
print(tested_y)

[3, 2, 3, 2, 5, 5, 1, 1, 2, 3, 4, 3, 4, 5, 0, 0, 4, 0, 0, 0, 5, 1, 5, 0, 1, 5, 1, 5, 5, 5, 1, 5]


## **Анализируем словарные толкования с помощью adagram**

In [8]:
with open('/content/drive/My Drive/Colab Notebooks/nlp/definitions_light.txt', 'r') as f2:
    definition = f2.read()
print(definition)

﻿http://feb-web.ru/feb/mas/mas-abc/default.asp


СВЕТ1, -а (-у), предл. в све́те, на свету́, м.

1. Электромагнитное излучение, воспринимаемое глазом и делающее видимым окружающий мир. Солнечный свет. Свет луны. Свет свечи. Луч света. Скорость света. Преломление света. Свет и тьма. □ Слабым светом горит ночник. Григорович, Гуттаперчевый мальчик. Алый свет вечерней зари медленно скользит по корням и стволам деревьев. Тургенев, Ермолай и мельничиха.

2. Освещение, характерное для какой-либо части суток. Дневной свет. □ Синими переливами играл утренний свет. Шолохов, Тихий Дон. На улицах был еще неопределенный, рассеянный свет белых ночей. Каверин, Открытая книга. || Разг. Рассвет, восход солнца. Еще семь часов до света! Тогда эта черная, полная мокрого снега ночь уйдет и даст место серому дню. Гаршин, Очень коротенький роман. Как и в прошлую ночь, едва забрезжил свет, она уже проснулась. Чехов, Невеста.

3. Источник освещения и приспособление для освещения в домах и на улицах. Зажечь в к

In [9]:
definitions = []
for line in definition.split('\n'):
  try:
      if line[0].isalnum() and line[1] == '.':
          defen = line.split('.')[1]
          definitions.append(defen)
  except:
      pass

definitions

[' Электромагнитное излучение, воспринимаемое глазом и делающее видимым окружающий мир',
 ' Освещение, характерное для какой-либо части суток',
 ' Источник освещения и приспособление для освещения в домах и на улицах',
 ' Место, откуда исходит освещение, освещенное место, пространство, где светло',
 ' Светлое место, пятно на картине, передающее наибольшую освещенность какого-л',
 ' Блеск глаз под влиянием какого-либо чувства, радостное, ясное выражение лица',
 ' То, что делает ясным, понятным мир, то, что делает радостной, счастливой жизнь',
 'Ласковое, приветливое обращение к кому-либо',
 ' Земля со всем существующим на ней, мир1, вселенная',
 ' Окружающие люди, общество',
 ' Ограниченный круг людей, составляющий высший слой привилегированных классов буржуазно-дворянского общества']

### **Посмотрим, как соотносит тип значения и словарные толкования adagram**

In [10]:
for defn in definitions:
    print(np.argmax(vm.disambiguate('свет', normalize_pm(defn))), defn)

0  Электромагнитное излучение, воспринимаемое глазом и делающее видимым окружающий мир
0  Освещение, характерное для какой-либо части суток
1  Источник освещения и приспособление для освещения в домах и на улицах
1  Место, откуда исходит освещение, освещенное место, пространство, где светло
0  Светлое место, пятно на картине, передающее наибольшую освещенность какого-л
4  Блеск глаз под влиянием какого-либо чувства, радостное, ясное выражение лица
2  То, что делает ясным, понятным мир, то, что делает радостной, счастливой жизнь
4 Ласковое, приветливое обращение к кому-либо
4  Земля со всем существующим на ней, мир1, вселенная
3  Окружающие люди, общество
3  Ограниченный круг людей, составляющий высший слой привилегированных классов буржуазно-дворянского общества


/usr/local/lib/python3.6/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [0]:
synsets = wikiwordnet.get_synsets('свет')

In [12]:
len(synsets)

8

In [13]:
for synset in synsets:
  print({w.definition() for w in synset.get_words()})

{'свет~ru~свет~ru~{{физ.|ru}} электромагнитное [[излучение]] в видимом для живых существ частотном [[диапазон]]е {{пример|Но как они ни старались, им не удалось разобрать ни одного слова, несмотря на яркий {{выдел|свет}} фонаря: они были неграмотны и не умели читать.|Л.\xa0А.\xa0Чарская|Король с раскрашенной картинки|1912|источник=НКРЯ}} {{пример|На площадке темно, и нигде нет намёка на {{выдел|свет}}, а то, что проносится перед глазами, бесформенно, мутно и непонятно.|Л.\xa0Н.\xa0Андреев|Вор|1904|источник=НКРЯ}}~69792~61748'}
{'свет~ru~свет~ru~[[источник]] света {{пример|И вдруг опал электрический {{выдел|свет}} ― остался только багрово-тусклый накалённый в лампочке волосок.|Солженицын|В круге первом, т.1|1968|источник=НКРЯ}} {{пример|Вот, вызывая тревогу, блеснул яркий солнечный {{выдел|свет}}, но навстречу снова надвигалась облачная стена.|И.\xa0А.\xa0Ефремов|Бухта радужных струй|1944|источник=НКРЯ}}~53885~62671'}
{'свет~ru~свет~ru~{{спец.|ru}}, {{мн.}} светлые [[участок|участки]] и

In [14]:
for idx in range(8):
    for hypernym in wikiwordnet.get_hypernyms(synsets[idx]):
        print(idx, {w.lemma() for w in hypernym.get_words()})

0 {'излучение'}
2 {'регион', 'участок', 'область', 'местность'}
5 {'период'}


In [15]:
for idx, sense in enumerate(vm.word_sense_probs('свет')):
    print(idx,[i[0] for i in vm.sense_neighbors('свет', sense[0])])

0 ['луч', 'световой', 'рассеивать', 'излучение', 'спектр', 'излучать', 'монохроматический', 'интенсивность', 'радиоволна', 'фотон']


/usr/local/lib/python3.6/dist-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


1 ['сияние', 'бледный', 'отблеск', 'неяркий', 'свечение', 'мерцание', 'отсвет', 'тусклый', 'блик', 'блик']
2 ['земля', 'похожий', 'походить', 'мир', 'планета', 'край', 'диво', 'сцена', 'способный', 'жениться']
3 ['появляться', 'переиздание', 'выходить', 'издание', 'книга', 'сборник', 'трехтомник', '1869', 'экран', 'журнал']
4 ['сияние', 'сиять', 'озарять', 'тьма', 'луч', 'истина', 'светоч', 'неизреченный', 'вечность', 'лучезарный']
5 ['лампа', 'ночник', 'погасить', 'зажигать', 'лампа', 'зажигаться', 'освещение', 'лампочка', 'дневной', 'фонарь']


Напомним, что мы выбрали значения для распознавания:

**1) Электромагнитное излучение и его источник**

**2) Окружающий нас мир, вселенная**

Для подходящих нам значений wikiwordnet выделил гипероним значения (1) - "излучение" - значение (0) по adagram. Однако к этой же группе значений можно отнести (1) значение по adagram; (4) и (5) значение по adagram соответствуют значению (3) в нашем списке, 

Если посмотреть на толкование (3) по adagram, то в соответствии с нашим противопоставлением двух значений отнесем его в группу значений (2), т.к. никакого отношения к излучению или источнику света оно не имеет

Таким образом, словарному **значению (1) соответствуют значения 0, 1; значению (3) -  4, 5 значения, выделяемые adagram, словарному значению (2) соответствует значение (2) по adagram**

Способы сопоставления значений по adagram и словарных толкований при помощи wikiwordnet:
* Для каждого значения, выделяемого wordnet, составляем список синсетов и гиперонимов (например, гиперонимами для слова свет в значении (1) могли бы являться "физическое явление", "волна", синсетами - "частицы", "преломление" и т.п. )
* Ищем пересечение полученных списков со словарным толкованием или примерами употребления данного значения в контексте по словарю

---
* Попытаемся реализовать просто пересечение слов контекста и толкований

In [16]:
cdef = [set(normalize_pm(definitions[0])),  set(normalize_pm(definitions[-2] + definitions[-3])), set(normalize_pm(definitions[2]))]
cdef

[{'видимый',
  'воспринимать',
  'глаз',
  'делать',
  'излучение',
  'мир',
  'окружающий',
  'электромагнитный'},
 {'весь',
  'вселенная',
  'земля',
  'мир1',
  'общество',
  'окружающий',
  'существующий',
  'человек'},
 {'дом', 'источник', 'освещение', 'приспособление', 'улица'}]

In [17]:
tested_y0 = []
for item in test_X:
    tmp = {idx: len(set(normalize_pm(item)) & cdef[idx]) for idx in range(3)}
    tmp = sorted(tmp.items(), key = lambda x: x[1], reverse=True)
    tested_y0.append(tmp[0][0] + 1)
print(tested_y0)
print(test_y)
print(metrics.f1_score(test_y, tested_y0, average='macro'))

[2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
0.2161172161172161


Как видим, получилось не очень хорошо. Нужно будет придумать другой способ, но wikiwordnet не дает нам много данных...

* **Составим словарь соотнесения значений**

In [0]:
values  = [1, 1, 2, 2, 3, 3] 

In [19]:
tested_y2 = [values[int(i)] for i in tested_y]
print(tested_y2)
print(test_y)

[2, 2, 2, 2, 3, 3, 1, 1, 2, 2, 3, 2, 3, 3, 1, 1, 3, 1, 1, 1, 3, 1, 3, 1, 1, 3, 1, 3, 3, 3, 1, 3]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


### **Оценка точности**

In [20]:
metrics.f1_score(test_y, tested_y2, average='macro')

0.5961352657004831

**Посмотрим, где же ошибки?**

In [21]:
print('true\tpredicted\ttext')
for x, y, z, w in zip(test_X, test_y, tested_y2, tested_y):
    if y != z:
        print('\t'.join([str(y),str(z)+' <- '+str(w), x]))
        
# значок стрелочки обозначает, что мы  некоторые значения, выделяемые adagram, объединяли в одно словарное значение 

true	predicted	text
2	3 <- 5	Ставши инженером, в чистое одетый, часто и при галстуке, я попадал на лекции и селекторные совещания в отделении дороги, где открыл, что более чванливого и спесивого народа, чем железнодорожное начальство, нет на всём белом свете.
2	3 <- 5	Милиционера КЛЕКОТОВА никто на белом свете не любил.
2	1 <- 1	То уверял, что египетская пирамида произошла из могильного кургана, причем четыре стены его были ориентированы сообразно четырем сторонам света.
2	1 <- 1	Горное эхо вторило грозе и разносило громовые удары « во все стороны света белого».
2	3 <- 4	"мирового пространства свет вечный ? вот что значит « по ту сторону»; «на том свете» ? понятие мистическое.
2	3 <- 4	Вследствие этого, Император Александр III пользовался, с одной стороны, общим доверием и уважением всех своих приближенных, а с другой стороны, что гораздо еще важнее, уважением и доверием всего света.
2	3 <- 5	Затем татары, крымцы, поляки, шведы ? нашим предкам приходилось отбиваться на все четыре сторо

* Видим, что в основном ошибочно было определено **значение (2)**, когда свет не имел отношения к излучению, а был употреблен в значении "весь мир, вселенная"
* Также ошибки заключались в том, что adagram неверно соотносил прямое и переносное значения: сам свет и источник света, и таким образом **путал значения (1) и (3)** 